# Notebook File Overview
This notebook features two different SQL queries based on the Study cohort we created (has diabetes) along with the parameters we set for the variables we selected from the concept sets and dataset builder step.

**The first SQL query extracts our demographic variables we chose**

**The second SQL query extracts the survey question variables we chose**

This *Study* Cohort should have the following characteristics:
* Age 25-50
* Race: White, Black, & Asian
* Ethnicity: Hispanic Yes or No
* Gender: Male or Female
* Condition: YES Diabetes
* PTSD survey questions: Currently seeing a doctor? & Currently prescribed meds?

Each query is autogenerated from the details we provided during the cohort builder and dataset builder phase which creates the tables based on the Concept Sets we selected:
1.	Demographics
2.	Survey

# SQL Code Chunk to Extract Demographic Variables

This part is an SQL query which extracts the demographic variables we selected in the dataset builder filtered and subsetted by the parameters we chose using the Cohort Builder:

* Age-group (25-50)
* Ethnicity (Hispanic Yes/No)
* Race (White, Black, or Asian)
* Gender (Male or Female)
* Inclusion criteria (we INCLUDE those with diabetes)

**The following code chunks perform three key steps:**

1. **Creates the SQL query as a text string** (does not execute the query yet)
2. **Creates the file path** and displays it
3. **Executes the BigQuery export** which runs the query and saves results as a CSV file to the specified path

In [2]:
library(tidyverse)
library(bigrquery)

# This query represents dataset "diabetes_study_ptsd" for domain "person" and was generated for All of Us Controlled Tier Dataset v8
# The paste() function probably has smart quotes. Try this version:
dataset_89304976_person_sql <- paste("
    SELECT
        person.person_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        p_race_concept.concept_name as race,
        p_ethnicity_concept.concept_name as ethnicity 
    FROM
        `person` person 
    LEFT JOIN
        `concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id  
    WHERE
        person.PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                person_id 
            FROM
                `cb_search_person` p 
            WHERE
                DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 25 AND 50 
                AND NOT EXISTS (      SELECT
                    'x'      
                FROM
                    `death` d      
                WHERE
                    d.person_id = p.person_id ) ) 
            AND cb_search_person.person_id IN (SELECT
                person_id 
            FROM
                `person` p 
            WHERE
                ethnicity_concept_id IN (38003564, 38003563) ) 
            AND cb_search_person.person_id IN (SELECT
                person_id 
            FROM
                `person` p 
            WHERE
                gender_concept_id IN (45878463, 45880669) ) 
            AND cb_search_person.person_id IN (SELECT
                person_id 
            FROM
                `person` p 
            WHERE
                race_concept_id IN (8527, 8516, 8515) ) 
            AND cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `cb_criteria` cr       
                        WHERE
                            concept_id IN (201820)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) )", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
person_89304976_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "person_89304976",
  "person_89304976_*.csv")
message(str_glue('The data will be written to {person_89304976_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_89304976_person_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  person_89304976_path,
  destination_format = "CSV")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
The data will be written to gs://fc-secure-ef51a80e-71e7-4b53-80c7-ed6ba790622a/bq_exports/maximilian.wegener@researchallofus.org/20251027/person_89304976/person_89304976_*.csv. Use this path when reading the data into your notebooks in the future.



In [3]:
# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {person_89304976_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(gender = col_character(), race = col_character(), ethnicity = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_89304976_person_df <- read_bq_export_from_workspace_bucket(person_89304976_path)

dim(dataset_89304976_person_df)

head(dataset_89304976_person_df, 5)

Loading gs://fc-secure-ef51a80e-71e7-4b53-80c7-ed6ba790622a/bq_exports/maximilian.wegener@researchallofus.org/20251027/person_89304976/person_89304976_000000000000.csv.



[1] 8142    5

person_id,gender,date_of_birth,race,ethnicity
<dbl>,<chr>,<chr>,<chr>,<chr>
1302288,Male,1976-06-15 00:00:00 UTC,Asian,Hispanic or Latino
1028513,Male,1977-06-15 00:00:00 UTC,Asian,Hispanic or Latino
8649818,Male,1977-06-15 00:00:00 UTC,Asian,Hispanic or Latino
1606071,Male,1996-06-15 00:00:00 UTC,Asian,Hispanic or Latino
2356675,Male,1982-06-15 00:00:00 UTC,Asian,Hispanic or Latino


# SQL Code Chunk to Extract Survey Variables

This SQL query extracts the survey variables we selected in the dataset builder concept sets:

* Survey question 1: Currently seeing a doctor?
* Survey question 2: Currently prescribed meds?

The results are filtered and subsetted by the parameters we chose using the Cohort Builder:

* Age group (25-50)
* Ethnicity (Hispanic Yes/No)
* Race (White, Black, or Asian)
* Gender (Male or Female)
* Inclusion criteria (**INCLUDES** those with diabetes)

**The following code chunks perform three key steps:**

1. **Creates the SQL query as a text string** (does not execute the query yet)
2. **Creates the file path** and displays it
3. **Executes the BigQuery export** which runs the query and saves results as a CSV file to the specified path

In [4]:
library(tidyverse)
library(bigrquery)

# This query represents dataset "diabetes_study_ptsd" for domain "survey" and was generated for All of Us Controlled Tier Dataset v8
dataset_89304976_survey_sql <- paste("
    SELECT
        answer.person_id,
        answer.question,
        answer.answer  
    FROM
        `ds_survey` answer   
    WHERE
        (
            question_concept_id IN (43528846, 43530361)
        )  
        AND (
            answer.PERSON_ID IN (SELECT
                distinct person_id  
            FROM
                `cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (SELECT
                    person_id 
                FROM
                    `cb_search_person` p 
                WHERE
                    DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 25 AND 50 
                    AND NOT EXISTS (      SELECT
                        'x'      
                    FROM
                        `death` d      
                    WHERE
                        d.person_id = p.person_id ) ) 
                AND cb_search_person.person_id IN (SELECT
                    person_id 
                FROM
                    `person` p 
                WHERE
                    ethnicity_concept_id IN (38003564, 38003563) ) 
                AND cb_search_person.person_id IN (SELECT
                    person_id 
                FROM
                    `person` p 
                WHERE
                    gender_concept_id IN (45878463, 45880669) ) 
                AND cb_search_person.person_id IN (SELECT
                    person_id 
                FROM
                    `person` p 
                WHERE
                    race_concept_id IN (8527, 8516, 8515) ) 
                AND cb_search_person.person_id IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `cb_search_all_events` 
                    WHERE
                        (concept_id IN(SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `cb_criteria` c 
                        JOIN
                            (SELECT
                                CAST(cr.id as string) AS id       
                            FROM
                                `cb_criteria` cr       
                            WHERE
                                concept_id IN (201820)       
                                AND full_text LIKE '%_rank1]%'      ) a 
                                ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                OR c.path LIKE CONCAT('%.', a.id) 
                                OR c.path LIKE CONCAT(a.id, '.%') 
                                OR c.path = a.id) 
                        WHERE
                            is_standard = 1 
                            AND is_selectable = 1) 
                        AND is_standard = 1 )) criteria ) )
        )", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
survey_89304976_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "survey_89304976",
  "survey_89304976_*.csv")
message(str_glue('The data will be written to {survey_89304976_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_89304976_survey_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  survey_89304976_path,
  destination_format = "CSV")

The data will be written to gs://fc-secure-ef51a80e-71e7-4b53-80c7-ed6ba790622a/bq_exports/maximilian.wegener@researchallofus.org/20251027/survey_89304976/survey_89304976_*.csv. Use this path when reading the data into your notebooks in the future.



In [5]:
# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {survey_89304976_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(survey = col_character(), question = col_character(), answer = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_89304976_survey_df <- read_bq_export_from_workspace_bucket(survey_89304976_path)

dim(dataset_89304976_survey_df)

head(dataset_89304976_survey_df, 5)

Loading gs://fc-secure-ef51a80e-71e7-4b53-80c7-ed6ba790622a/bq_exports/maximilian.wegener@researchallofus.org/20251027/survey_89304976/survey_89304976_000000000000.csv.

Warning message:
“The following named parsers don't match the column names: survey”


[1] 1318    3

person_id,question,answer
<dbl>,<chr>,<chr>
5785022,Are you still seeing a doctor or health care provider for post-traumatic stress disorder (PTSD)?,Are you still seeing a doctor or health care provider for post-traumatic stress disorder (PTSD)? - Yes
1236209,Are you still seeing a doctor or health care provider for post-traumatic stress disorder (PTSD)?,Are you still seeing a doctor or health care provider for post-traumatic stress disorder (PTSD)? - Yes
2615228,Are you currently prescribed medications and/or receiving treatment for post-traumatic stress disorder (PTSD)?,Are you currently prescribed medications and/or receiving treatment for post-traumatic stress disorder (PTSD)? - No
2437043,Are you currently prescribed medications and/or receiving treatment for post-traumatic stress disorder (PTSD)?,Are you currently prescribed medications and/or receiving treatment for post-traumatic stress disorder (PTSD)? - No
6083377,Are you currently prescribed medications and/or receiving treatment for post-traumatic stress disorder (PTSD)?,Are you currently prescribed medications and/or receiving treatment for post-traumatic stress disorder (PTSD)? - Yes


# Let's clean and join our data before saving to our workspace bucket

**Step 1: Sort our survey data by *person_id***

Create a new object called survey_arranged using the assignment (<-) operator

Then, we will use the *arrange()* function from the **Dplyr** package within the **Tidyverse** (loaded earlier)

In [7]:
survey_arranged <- dataset_89304976_survey_df %>%
arrange(person_id)

dim(survey_arranged)
head(survey_arranged, 5)

[1] 1318    3

person_id,question,answer
<dbl>,<chr>,<chr>
1011367,Are you still seeing a doctor or health care provider for post-traumatic stress disorder (PTSD)?,Are you still seeing a doctor or health care provider for post-traumatic stress disorder (PTSD)? - Yes
1011367,Are you currently prescribed medications and/or receiving treatment for post-traumatic stress disorder (PTSD)?,Are you currently prescribed medications and/or receiving treatment for post-traumatic stress disorder (PTSD)? - No
1014304,Are you currently prescribed medications and/or receiving treatment for post-traumatic stress disorder (PTSD)?,Are you currently prescribed medications and/or receiving treatment for post-traumatic stress disorder (PTSD)? - Yes
1014304,Are you still seeing a doctor or health care provider for post-traumatic stress disorder (PTSD)?,Are you still seeing a doctor or health care provider for post-traumatic stress disorder (PTSD)? - Yes
1015044,Are you still seeing a doctor or health care provider for post-traumatic stress disorder (PTSD)?,Are you still seeing a doctor or health care provider for post-traumatic stress disorder (PTSD)? - No


**Step 2: Clean up the answer column**

We will also create a new object called survey_clean which is modifies the previous survey_arranged data frame

As you can see in the previous chunk, the answer column includes the same string of text found in the question column which we want to **remove**

We will use the *str_remove()* and *str_trim()* functions from the **stringr** package within the **Tidyverse**

In [8]:
survey_clean <- survey_arranged %>%
  mutate(
    answer = str_remove(answer, fixed(question)), # remove the exact question text (literal match)
    answer = str_remove(answer, "^\\s*-\\s*"),    # remove a leading hyphen and surrounding spaces left behind
    answer = str_trim(answer)                     # trim any leftover whitespace
  )

head(survey_clean)

person_id,question,answer
<dbl>,<chr>,<chr>
1011367,Are you still seeing a doctor or health care provider for post-traumatic stress disorder (PTSD)?,Yes
1011367,Are you currently prescribed medications and/or receiving treatment for post-traumatic stress disorder (PTSD)?,No
1014304,Are you currently prescribed medications and/or receiving treatment for post-traumatic stress disorder (PTSD)?,Yes
1014304,Are you still seeing a doctor or health care provider for post-traumatic stress disorder (PTSD)?,Yes
1015044,Are you still seeing a doctor or health care provider for post-traumatic stress disorder (PTSD)?,No
1015044,Are you currently prescribed medications and/or receiving treatment for post-traumatic stress disorder (PTSD)?,No


**Step 3: Make the values in the question column the column names and the values in the answer column the values for the new columns**

This will get rid of the person_id duplicates but keep both the questions and their responses

We will create a new object called survey_wide which is modifies the previous survey_clean data frame

We will use the *pivot_wider()* function from the **tidyr** package within the **Tidyverse** to accomplish this

In [9]:
survey_wide <- survey_clean %>%
  pivot_wider(
    names_from = question,
    values_from = answer
  )

dim(survey_wide)
head(survey_wide, 5)

[1] 659   3

person_id,Are you still seeing a doctor or health care provider for post-traumatic stress disorder (PTSD)?,Are you currently prescribed medications and/or receiving treatment for post-traumatic stress disorder (PTSD)?
<dbl>,<chr>,<chr>
1011367,Yes,No
1014304,Yes,Yes
1015044,No,No
1017367,Yes,Yes
1021985,No,No


**Step 4: Join the updated survey data frame, survey_wide, to our demographic variable data frame *dataset_89304976_person_df***

We will create a new object called diabetes_study_ptsd which will include variables from our survey and demographic variable data frames.

We will use the *left_join()* function from the **dplyr** package to join the two data frames on the **person_id** variable.

We will also use the *filter()* function from the **dplyr** package to remove NA values from our new data frame.

In [11]:
library(dplyr)

diabetes_study_ptsd <- dataset_89304976_person_df %>%
  left_join(survey_wide, by = c("person_id" = "person_id")) %>%
  filter(
    !is.na(`Are you still seeing a doctor or health care provider for post-traumatic stress disorder (PTSD)?`),
    !is.na(`Are you currently prescribed medications and/or receiving treatment for post-traumatic stress disorder (PTSD)?`)
  )

dim(diabetes_study_ptsd)
head(diabetes_study_ptsd, 5)

[1] 659   7

person_id,gender,date_of_birth,race,ethnicity,Are you still seeing a doctor or health care provider for post-traumatic stress disorder (PTSD)?,Are you currently prescribed medications and/or receiving treatment for post-traumatic stress disorder (PTSD)?
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1302288,Male,1976-06-15 00:00:00 UTC,Asian,Hispanic or Latino,Yes,Yes
2000298,Male,1977-06-15 00:00:00 UTC,Asian,Hispanic or Latino,Yes,Yes
1913213,Female,1986-06-15 00:00:00 UTC,Black or African American,Hispanic or Latino,Yes,No
7661404,Female,1984-06-15 00:00:00 UTC,Black or African American,Hispanic or Latino,Yes,Yes
2684487,Female,1991-06-15 00:00:00 UTC,Black or African American,Hispanic or Latino,No,No


# Run the provided snippets to save our cleaned data frame to our workspace bucket

This section saves R data frames that are already in memory to our workspace bucket using a two-step process:

1. Writes dataframe to local CSV file on persistent disk
2. Copies file from local storage to workspace bucket using gsutil

It also verifies successful upload by listing bucket contents

Once our final data are saved to the workspace bucket, we no longer need our persistent disk

**MAY HAVE TO RUN SET UP SNIPPET FIRST BUT ITS JUST TO LOAD TIDYVERSE WHICH WE DID ALREADY**

In [12]:
# This snippet assumes that you run setup first

# This code saves your dataframe into a csv file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe <- diabetes_study_ptsd

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename <- 'diabetes_study_ptsd.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# store the dataframe in current workspace
write_excel_csv(my_dataframe, destination_filename)

# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ./", destination_filename, " ", my_bucket, "/data/"), intern=T)

# Check if file is in the bucket
system(paste0("gsutil ls ", my_bucket, "/data/*.csv"), intern=T)

character(0)

[1] "gs://fc-secure-ef51a80e-71e7-4b53-80c7-ed6ba790622a/data/diabetes_control_ptsd.csv"
[2] "gs://fc-secure-ef51a80e-71e7-4b53-80c7-ed6ba790622a/data/diabetes_study_ptsd.csv"  
[3] "gs://fc-secure-ef51a80e-71e7-4b53-80c7-ed6ba790622a/data/final_diabetes_ptsd.csv"